In [388]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import textwrap
import collections
import datetime as dt

In [390]:
p = PlotConstructor()
p.get_figure_size(1)

eacolours = EAColours()

In [160]:
class EAColours:
    
    def get_dwcolors(self, as_rgblist=False, subset=None):
        """
        Options for subset: 'EA', 'FUEL', 'HEAT7', 'ISLAND', 'ISLAND_1', 
                            'LINE', 'NZ', 'OPEN_INTEREST', 'ZONE'
        """
        from ast import literal_eval as make_tuple
        colours = pd.read_csv(self.root+r"colors\dwcolors.csv", index_col='Unnamed: 0')    
        colours['RGB'] = colours['RGB'].apply(lambda x: make_tuple(x))    
        if as_rgblist:
            return colours['RGB'].tolist()
        else:
            return colours

    def get_eacolors(self, sequence_list=None, as_rgblist=False):
        from ast import literal_eval as make_tuple
        eacolors_df = pd.read_csv(self.root+r"\colors\eacolors.csv", index_col='Unnamed: 0')
        eacolors_df['RGB'] = eacolors_df['RGB'].apply(lambda x: make_tuple(x))    
        if as_rgblist:
            return eacolors_df['RGB'].tolist()
        else:
            return eacolors_df    

    def get_color_list(self, colors=None, subset=None):
        """ Function for getting specific color lists """
        if colors == 'default':
            return self.get_eacolors(as_rgblist=True)    
        elif colors == 'dw':
            return self.get_dwcolors(as_rgblist=True, subset=subset)
        else:
            return colors    

    def get_custom_eacolors(self, colorlist, reset_index=False):    
        """ e.g. get_custom_eacolors(['red','red','powdersnow'], reset_index=True) """
        df = self.get_eacolors()   
        df = df.ix[colorlist]
        if reset_index:
            df['Sequence'] = range(0,len(df))
        return df

    def __init__(self):        
        self.root = os.getcwd()

In [583]:
class PlotConstructor:
    
    def apply_rc_params(self, style):        

        
        fontsize = {1: 11, 2: 11, 3: 9.5, 4: 11, 5: 11}
        
        import matplotlib as mpl
        font = {'family' : 'Arial',  
                'size'   : fontsize[style],   
                'weight' : 100
                }   
        axes = {'axisbelow' : True,   
                'linewidth' : 0.6,    
                'edgecolor' : 'black',
                'facecolor' : 'white',
                }              
        lines = {'linewidth' : 1.5
                 }
        legend = {'fontsize' : fontsize[style]
                  }    
        mpl.rc('font', **font) 
        mpl.rc('axes', **axes)
        mpl.rc('lines', **lines)
        mpl.rc('legend', **legend)        
    
    
        """
        if self.style in [3]:
            
            # Clear font cache
            import matplotlib as mpl
            for x in os.listdir(mpl.get_cachedir()):
                try:
                    os.remove(os.path.join(mpl.get_cachedir(),x))
                except:
                    pass
            import matplotlib.font_manager as font_manager
            path = 'fonts//helvetica-neue-light.ttf'
            prop = font_manager.FontProperties(fname=path)
            print("Applying Heveltica Neue Light")
            mpl.rcParams['font.family'] = prop.get_name()
        else:
            mpl.rcParams['font.family'] = 'Arial'
        """
    
    def get_figure_size(self,fig_size):
        dimensions={1:(18.5,10.5),2:(25,13),3:(25,13),4:(18,9),5:(10.5,5.68),6:(21,5.68),7:(21,22.74)}[fig_size] 
        return tuple(x/2.54 for x in dimensions)
    
    
    def print_figure_size_description(self):
        print("Figure size 1 is for word portrait")
        print("Figure size 2 is for word landscape")
        print("Figure size 3 is for PowerPoint")
        print("Figure size 4 is the standard size for print publishing (WMN)")
        print("Figure size 5 is the standard plot size for Tuong's report")
        print("Figure size 6 is for the HVDC plot in Tuong's report")
        print("Figure size 7 is the size of the report binding constraint table")
        
    
    def print_style_description(self):
        print("Style 1 is Matlab techincal")
        print("Style 2 is Matlab print publishing")
        print("Style 3 is EMI web publishing")
        print("Style 4 is EMI technical")
        print("Style 5 is EMI detail (print publishing)")
    
    
    def apply_style(self):
        print('Applying style')
                
        fig=self.fig

        axes = [self.ax]
        
        if self.second_y_columns:            
            self.ax2 = self.ax.twinx()
            axes.append(self.ax2)
        
        self.apply_rc_params(self.style)
        
        for ax in axes:
            # Spines
            if self.style in [2,3,5]:
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                ax.spines['left'].set_visible(False)            

            # Gridlines
            if self.style in [1,4]:            
                plt.grid(b=True,which='major',axis='both',linestyle=':',alpha=1,color=[0.501,0.502,0.503],linewidth=1)   
            elif self.style in [2,3,5]:            
                plt.grid(b=True,which='major',axis='y',linestyle='-',alpha=1,color=[0,0,0],linewidth=0.15)             
                ax.yaxis.set_ticks_position('none') #set where y axis ticks are drawn (left, right, both or none)

            # Ticks
            ax.tick_params('both', length=0, width=0.6, which='major')
            if self.style in [2,3,5]:
                for tick in ax.xaxis.get_major_ticks():
                    tick.tick1line.set_markersize(-4) #bottom
                    tick.tick2line.set_markersize(0) #top

                #ax.xaxis.set_tick_params(size=4,width=0.6,direction='out') 

            # Background transparency
            if self.style == 3:
                fig.patch.set_alpha(1)
                fig.tight_layout()
                fig.subplots_adjust(left=0.1, top=0.9, bottom=0.1, right=0.825)
            else:
                fig.patch.set_alpha(0)
                plt.subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9)


    
    
    def init_plot(self):
        print('Initiating plot')
        self.fig = plt.figure(figsize=self.get_figure_size(self.fig_size)) #plt.figure creates a new blank figure
        self.ax = self.fig.add_subplot(111)  #Creates the subplot
        

        
        self.apply_style() #This applies examiner's EA styling to the plot   
    
    def save_plot(self):
        bbox_inches = {1:'tight',2:'tight',3:None,4:'tight',5:'tight'}         
        if self.style in [3]:
            pad_inches = 0.4
        else:
            pad_inches=0.1
        print('Saving plot');print()
        plt.savefig(self.filename, bbox_inches=bbox_inches[self.style], pad_inches=pad_inches, dpi=900)
    
    def __init__(self):
        pass    




class Plot(PlotConstructor):
    
    def draw_secondary(self):
        pass
    
    def draw(self):
        pass   
    
    def __init__(self, data, fig_size=2, style=2, filename='default', file_format='.png', x_label='', y_label='',
                 plot_colours=None, 
                 x_tick_rotation=0, x_tick_datefmt=None,
                 second_y_kind='line', second_y_columns=[], second_y_label='', second_y_min=None, second_y_max=None,
                 legend_wrap_len=20, reverse_legend=False, sort_legend=True, custom_leg_x_offset=0, legend_position='best',
                 marker='', ms=0, linestyle='-',lw=1.8, markeredgewidth=0,
                 save=False ):
        
        self.data = data
        
        #self.is_dates = data.index.is_all_dates 
        
        self.columns = data.columns.tolist()        
        
        self.fig_size = fig_size
        self.style = style     
        self.x_label = x_label
        self.y_label = y_label
        
        self.x_tick_rotation = x_tick_rotation
        self.x_tick_datefmt = x_tick_datefmt        
        
        self.main_y_columns = [x for x in self.columns if x not in second_y_columns]
        self.second_y_columns = [x for x in self.columns if x in second_y_columns]
        self.second_y_kind = second_y_kind
        self.second_y_label = second_y_label
        self.second_y_min = second_y_min
        self.second_y_max = second_y_max   

        self.legend_wrap_len = legend_wrap_len
        self.reverse_legend = reverse_legend
        self.sort_legend = sort_legend
        self.custom_leg_x_offset = custom_leg_x_offset #something close to ~0.1 is recommended for a secondary Y plot
        self.legend_position = legend_position

        if plot_colours is None:
            eacolours = EAColours()
            plot_colours = eacolours.get_color_list(colors='default')
        self.plot_colours = plot_colours
        
        self.marker = marker
        self.ms = ms
        self.linestyle = linestyle
        self.lw = lw
        self.markeredgewidth = markeredgewidth
    
        self.filename = filename
    
        # Draw the plot
        self.init_plot()        
                        
        self.draw()
        
        self.ax.set_xlabel(x_label)
        self.ax.set_ylabel(y_label)
        if self.second_y_columns and self.second_y_label:
            self.ax2.set_ylabel(second_y_label)
            
        #plt.xticks(rotation=x_tick_rotation)        
        plt.setp( self.ax.xaxis.get_majorticklabels(), rotation=x_tick_rotation, horizontalalignment='center')
        
        self.save_plot()        
            
            
        
    
class Bar(Plot):    
    
    def draw(self):
        print('Drawing bar')        

class Area(Plot):    
    def cumsum_data(self):
        print('Cumsumming')
    
    def draw(self):        
        self.cumsum_data() 
        print('Drawing area')
        
        # Plot the area first
        
        # Plot secondary 

class Line(Plot):    
    
    def configure_legend(self):
            
        for c in self.main_y_columns:
            self.plot_controller[c]['handle']= plt.Line2D((0,1),(0,0), 
                                                          color=self.plot_controller[c]['color'],
                                                          marker=self.plot_controller[c]['marker'],
                                                          ms=self.plot_controller[c]['ms'],
                                                          lw=self.plot_controller[c]['lw'],
                                                          linestyle=self.plot_controller[c]['linestyle'],
                                                          markeredgewidth=self.plot_controller[c]['markeredgewidth']
                                                         )            
        for c in self.second_y_columns:            
            if self.second_y_kind in ['bar','area']:                
                self.plot_controller[c]['handle'] = plt.Rectangle((0,0),1,1, fc=self.plot_controller[c]['color'], lw=0)
            else:
                self.plot_controller[c]['handle'] = plt.Line2D((0,1),(0,0),
                                                               color=self.plot_controller[c]['color'],
                                                               marker=self.plot_controller[c]['marker'],
                                                               ms=self.plot_controller[c]['ms'],
                                                               lw=self.plot_controller[c]['lw'],
                                                               linestyle=self.plot_controller[c]['linestyle'],
                                                               markeredgewidth=self.plot_controller[c]['markeredgewidth']
                                                              )
        
        
    def draw_legend(self):                
        
        if self.sort_legend:
            dfp=self.data[self.main_y_columns]
            dfp_list=dfp.ix[dfp.index[-1]].sort_values(ascending=False).index.tolist()
            dfs=self.data[self.second_y_columns]
            dfs_list=dfs.ix[dfs.index[-1]].sort_values(ascending=False).index.tolist()           
            legend_list = dfp_list+dfs_list
        else:
            legend_list = self.main_y_columns + self.second_y_columns

        lns = [self.plot_controller[x]['handle'] for x in legend_list]
        labs = [self.plot_controller[x]['label'] for x in legend_list]
        
        #if self.reverse_legend:
        #    lns=lns[::-1]
        #    labs=labs[::-1]   
        
        if self.style in [1,4]:
            plt.legend(lns,labs,
                       loc=self.legend_position,
                       numpoints=1).get_frame().set_linewidth(0.6)    
            
        elif self.style in [2,5]:    
            plt.legend(lns,labs,
                       bbox_to_anchor=(1+self.custom_leg_x_offset, 1),
                       loc=2,
                       borderaxespad=0.,
                       framealpha=0,
                       numpoints=1).get_frame().set_linewidth(0)  
            
        elif self.style in [3]:    
            # place legend y in the middle
            plt.legend(lns,labs,
                       bbox_to_anchor=(1+self.custom_leg_x_offset, 1),
                       loc=2,
                       borderaxespad=0.,
                       framealpha=0,
                       numpoints=1).get_frame().set_linewidth(0)   
            
            
    
    def configure_plot(self):
        """ Setup the plot dictionary """
        
        plot_controller = collections.OrderedDict()   
        for i,c in enumerate( self.main_y_columns  ):
            plot_controller[c] = {}
            plot_controller[c]['label'] = textwrap.fill(c.replace('_',' '), self.legend_wrap_len)
            plot_controller[c]['color'] = self.plot_colours[i]
            plot_controller[c]['marker'] = self.marker
            plot_controller[c]['ms'] = self.ms #markersize
            plot_controller[c]['linestyle'] = self.linestyle
            plot_controller[c]['lw'] = self.lw #linewidth
            plot_controller[c]['markeredgewidth'] = self.markeredgewidth 
            
            #plot_controller[column]['order'] = i            
            
        for j,c in enumerate( self.second_y_columns ):
            plot_controller[c] = {}
            plot_controller[c]['label'] = textwrap.fill(c.replace('_',' '), self.legend_wrap_len)
            plot_controller[c]['color'] = self.plot_colours[i+j+1]
            plot_controller[c]['marker'] = self.marker
            plot_controller[c]['ms'] = self.ms
            plot_controller[c]['linestyle'] = self.linestyle
            plot_controller[c]['lw'] = self.lw
            plot_controller[c]['markeredgewidth'] = self.markeredgewidth   
        self.plot_controller = plot_controller
        
        
    """
        #def configure_legend(self):
                if self.second_y_kind in ['bar','area']:                
                    plot_controller[c]['handle'] = plt.Rectangle((0,0),1,1 fc=plot_controller[c]['color'], lw=0)
                else:
                    plot_controller[c]['handle'] = plt.Line2D((0,1),(0,0), color=plot_controller[c]['color'],
                                                              marker=plot_controller[c]['marker'],
                                                              ms=plot_controller[c]['ms'],
                                                              lw=plot_controller[c]['lw'],
                                                              linestyle=plot_controller[c]['linestyle'],
                                                              markeredgewith=plot_controller[c]['markeredgewith']
                                                             )


                #plot_controller[column]['order'] = i+j+1
    """

    def redraw(self):
        self.draw(redraw=True)
        self.save_plot()

    def draw(self, redraw=False):
        if self.data.index.is_all_dates:
            print('This is a date plot')
            
            
        # Plot rules:
        
        # Secondary axes are plotted behind, therefore they are done first, except for Area, where it is plotted on top
        
        # ie. if Area, plot secondary last
        # ie.
        
        # Secondary axes colours take colours after primary axes colours are used up
        # Secondary axes legend labels/handles appear after primary axes legend labels/handles are used up
        
        if not redraw:
            self.configure_plot()
        self.configure_legend() 
        self.draw_legend()
            
        # Plot second 
        for column in self.second_y_columns:
            
            #print(self.data.index)
            #print(self.data[column])
            #print(self.plot_colours[i])
            #print(label)      

            if self.second_y_kind in ['bar']:
                self.ax2.bar(self.data.index, self.data[column],
                            color=self.plot_controller[column]['color'])
            else:        
                self.ax2.plot(self.data.index, self.data[column],
                              color=self.plot_controller[column]['color'], 
                              label=self.plot_controller[column]['label'],
                              marker=self.marker, ms=self.ms, lw=self.lw, markeredgewidth=self.markeredgewidth)

        # Plot main 
        for column in self.main_y_columns:
            
            #print(self.data.index)
            #print(self.data[column])
            #print(self.plot_colours[i])
            #print(label)            
            self.ax.plot(self.data.index, self.data[column],
                         color=self.plot_controller[column]['color'],
                         label=self.plot_controller[column]['label'],                         
                         marker=self.plot_controller[column]['marker'], 
                         ms=self.ms, lw=self.lw, markeredgewidth=self.markeredgewidth)
            
        print('Drawing line')

In [587]:
import numpy
df=pd.DataFrame(columns=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o'])
for i,d in enumerate(df.columns):
    df[d] = pd.Series(numpy.arange(0,10)+i)
    
for sc in ['a','e','i','o']:
    df[sc] = df[sc]*-10
df    


p = Line(df, style=1, second_y_columns=['a'], second_y_kind='bar', y_label='Main Y', second_y_label='Second Y',
         marker='o', ms=10, x_tick_rotation=0, save=True, custom_leg_x_offset=0.1, filename='default2')

p.plot_controller['a']['color'] = (0,0,0)
p.redraw()

Initiating plot
Applying style
Drawing line
Saving plot

Drawing line
Saving plot



In [588]:
p.main_y_columns + p.second_y_columns

['b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'a']

In [589]:
dfp=df[p.main_y_columns]
dfp_list=dfp.ix[dfp.index[-1]].sort_values(ascending=False).index.tolist()

dfs=df[p.second_y_columns]
dfs_list=dfs.ix[dfs.index[-1]].sort_values(ascending=False).index.tolist()

dfp_list+dfs_list

['n', 'm', 'l', 'k', 'j', 'h', 'g', 'f', 'd', 'c', 'b', 'e', 'i', 'o', 'a']

In [590]:
d = pd.read_excel("QRSS March 2016.xlsx",sheetname="Annual Residential Elec Cost",
                  skiprows=14,skip_footer=34,index_col=3).dropna(how="all")[['c/kWh','%','c/kWh.1','%.1','c/kWh.2','%.2']]
d.columns = [x.replace('.1',' (lines component)').replace('.2',' (energy and other component)') for x in d.columns.tolist()]

d.index = d.index.map(lambda x: dt.datetime( int(x), 1, 1))
#d.index = d.index.map(lambda x: dt.datetime( 2016, 1, 2017 - int(x) ))

In [380]:
#p1 = Bar(d, style=1, filename='style1.png')

p2 = Line(d[['c/kWh','%']].dropna(), style=2, filename='style2.png', x_label='Test', y_label='Test', x_tick_rotation=0,
         second_y_columns=['%'], second_y_label='Test 2')

#p3 = Line(d, style=3, filename='style3.png')
#p4 = Line(d, style=4, filename='style4.png')
#p5 = Area(d, style=5, filename='style5.png')

Initiating plot
Applying style
This is a date plot
Drawing line
Saving plot



C:\Anaconda\envs\anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [586]:
plt.show()

In [ ]:
p1.data.index.is_all_dates

In [ ]:
# Plot combos:
kind='area'
lines=[] #excludes these from the default type and plots as line(s) (EG if kind='area', columns her)
bars=[] #excludes these from the default type and plots as bar(s)
bar_kind='grouped' #grouped or stacked
secondary_y = #series to be plotted on the secondary y axis